In [2]:
import os
import pandas as pd
import pickle
from datetime import date

# =====================================================================
# 0️⃣ PATHS AND SETUP
# =====================================================================

eval_dir = "./cfpr_evaluation_dicts"
fig_dir = "./cfpr_figures_percent_error_dynamic"
report_dir = "./cfpr_reports"
os.makedirs(report_dir, exist_ok=True)

report_path = os.path.join(report_dir, f"cfpr_forecast_eval_report_{date.today()}.md")

# =====================================================================
# 1️⃣ LOAD ALL EVALUATION FILES
# =====================================================================

print("📦 Loading saved evaluation files...")

# Monthly evaluation
with open(os.path.join(eval_dir, "category_results_mape.pkl"), "rb") as f:
    category_results_mape = pickle.load(f)
monthly_summary = pd.read_csv(os.path.join(eval_dir, "overall_summary_mape.csv"))

# YoY evaluation
with open(os.path.join(eval_dir, "category_residual_results.pkl"), "rb") as f:
    category_residual_results = pickle.load(f)
yoy_summary = pd.read_csv(os.path.join(eval_dir, "overall_summary_yoy.csv"))

# Cross-analysis
with open(os.path.join(eval_dir, "combined_summaries.pkl"), "rb") as f:
    combined_summaries = pickle.load(f)
overall_cross = pd.read_csv(os.path.join(eval_dir, "overall_summary_cross.csv"))
coverage_cross = pd.read_csv(os.path.join(eval_dir, "model_coverage_cross.csv"))

print("✅ All data loaded successfully!\n")

# =====================================================================
# 2️⃣ BUILD MARKDOWN REPORT
# =====================================================================

with open(report_path, "w", encoding="utf-8") as f:
    # === HEADER ===
    f.write(f"# 📊 CFPR 2025 Forecast Evaluation Report\n")
    f.write(f"**Generated:** {date.today().strftime('%B %d, %Y')}\n\n")
    f.write("This report summarizes model performance across **monthly (MAPE)**, "
            "**year-over-year (YoY)**, and **cross-metric** evaluations for all food categories.\n\n")
    f.write("---\n\n")

    # =====================================================================
    # PER-CATEGORY SECTION
    # =====================================================================
    f.write("## 🍎 Category-Level Evaluations\n\n")

    for category in sorted(category_results_mape.keys()):
        f.write(f"### {category}\n\n")

        # --- 1️⃣ Combine MAPE + YoY ---
        mape_series = category_results_mape[category].mean().rename("MAPE (%)")
        yoy_series = category_residual_results.get(category)
        if yoy_series is not None:
            yoy_series = yoy_series.rename("YoY Residual (%)")
            combined = pd.concat([mape_series, yoy_series], axis=1).dropna().sort_values("MAPE (%)").head(10)
            f.write("**Top 10 Models (MAPE + YoY Residual)**\n\n")
            f.write(combined.round(2).to_markdown())
            f.write("\n\n")
        else:
            f.write("_No YoY data available for this category._\n\n")

        # --- 2️⃣ Add Figure ---
        img_name = f"{category.replace('/', '_').replace(' ', '_')}_percent_error_dynamic.png"
        img_path = os.path.join(fig_dir, img_name)
        if os.path.exists(img_path):
            f.write(f"**Monthly Percent Error Trends (Top 5 Models)**\n\n")
            f.write(f"![{category}]({img_path})\n\n")

        # --- 3️⃣ Recommended Models ---
        if category in combined_summaries:
            best_models = combined_summaries[category].head(3).index.tolist()
            f.write("**Recommended Models:** " + ", ".join(best_models) + "\n\n")

        f.write("---\n\n")

    # =====================================================================
    # OVERALL SECTION
    # =====================================================================
    f.write("## 🌐 Overall Model Performance Summary\n\n")

    f.write("### Global Ranking (Combined MAPE + YoY Residual)\n\n")
    f.write(overall_cross.head(15).round(3).to_markdown(index=False))
    f.write("\n\n")

    f.write("### Model Coverage (Number of Categories where Top-3)\n\n")
    f.write(coverage_cross.head(10).round(3).to_markdown(index=False))
    f.write("\n\n")

    top_models = overall_cross.head(5)["model"].tolist()
    f.write(f"### 🧠 Recommended Core Models (Across All Food Categories)\n\n")
    f.write("- " + "\n- ".join(top_models))
    f.write("\n\n")

print(f"✅ Report generated: {report_path}")


📦 Loading saved evaluation files...
✅ All data loaded successfully!

✅ Report generated: ./cfpr_reports/cfpr_forecast_eval_report_2025-10-21.md
